### Query Decomposition:

Query decomposition is the process of takinga complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

It is useful in
* Complex queries often involve multiple concepts.
* LLMs or retrievers may miss parts of the original question.
* It enables multi-hop reasoning (answering in steps)
* Allows parallelism(especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

d:\RAGs In Depth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#Data Ingestion and Embedding Documents

documents = TextLoader('langchain_crewai.txt', encoding="utf-8").load()

chunks = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50).split_documents(documents)

embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

vectorstore= FAISS.from_documents(chunks, embeddings)

retriever= vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":4, "lambda_mult":0.7})

#The lambda_mult parameter controls this trade-off:

# lambda_mult = 1.0 → prioritize only relevance (like normal similarity search)
# lambda_mult = 0.0 → prioritize only diversity

In [9]:
#llm model
llm = init_chat_model(model="groq:llama-3.1-8b-instant")
llm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000223B6EF8250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000223B84806D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
#query decomposition prompt

decompose_prompt = PromptTemplate.from_template(
    '''
    You are an AI Assistant. Decompose the following complex question into 2 or more sub-questions for better document retrieval.
    question: {question}

    sub-question: 
    '''
)

decomposition_chain = decompose_prompt | llm | StrOutputParser()

decomposition_chain

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n    You are an AI Assistant. Decompose the following complex question into 2 or more sub-questions for better document retrieval.\n    question: {question}\n\n    sub-question: \n    ')
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000223B6EF8250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000223B84806D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [11]:
query = "How does langchain use memory and agents compared to crewai?"
sub_questions= decomposition_chain.invoke({"question": query})
sub_questions

"To better document retrieval, I'll decompose the complex question into the following sub-questions:\n\n1. **What is LangChain and its core architecture?**\n\n   - This sub-question aims to understand the fundamentals of LangChain and its memory and agent components, which will provide context for the comparison.\n\n2. **How does LangChain utilize memory (e.g., LLMs, databases) and agent (e.g., decision-making, interaction) capabilities?**\n\n   - This sub-question will focus on the specific features and mechanisms LangChain employs for memory and agent management.\n\n3. **What are the key features of Crew AI and its approach to memory and agent management?**\n\n   - This sub-question will gather information on Crew AI's architecture, strengths, and weaknesses, which will facilitate a comparison with LangChain.\n\n4. **How do LangChain and Crew AI differ in their use of memory and agents, and what are the implications for application development and user experience?**\n\n   - This sub-